In [2]:
df = spark.read.csv('Jan_2020_ontime.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEP_TIME: integer (nullable = true)
 |-- DEP_DEL15: double (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- ARR_DEL15: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- _c21: string (nullable = true)



In [3]:
df_vertices = df.select('ORIGIN', 'ORIGIN_AIRPORT_ID').dropDuplicates(['ORIGIN']).withColumnRenamed('ORIGIN', 'id')

In [4]:
df_edges = df.withColumnRenamed('ORIGIN', 'src').withColumnRenamed('DEST', 'dst')

In [5]:
from graphframes import *

g = GraphFrame(df_vertices, df_edges)

/opt/spark/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o65.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.

In [9]:
factor_more_than_1 = g.edges.filter('DEP_DEL15 == 1')
df3 = factor_more_than_1.select('src', 'DEP_DEL15').distinct().orderBy('src', ascending=False)
factor_more_than_12 = g.edges.filter('ARR_DEL15 == 1')
df4 = factor_more_than_12.select('dst', 'ARR_DEL15').distinct().orderBy('dst', ascending=False)

In [10]:
inner = df4.join(df3, df3.src == df4.dst)
inner.show()

[Stage 10:===================================================>  (190 + 9) / 200]

+---+---------+---+---------+
|dst|ARR_DEL15|src|DEP_DEL15|
+---+---------+---+---------+
|BGM|      1.0|BGM|      1.0|
|INL|      1.0|INL|      1.0|
|PSE|      1.0|PSE|      1.0|
|MSY|      1.0|MSY|      1.0|
|PPG|      1.0|PPG|      1.0|
|DRT|      1.0|DRT|      1.0|
|GEG|      1.0|GEG|      1.0|
|BUR|      1.0|BUR|      1.0|
|SNA|      1.0|SNA|      1.0|
|GRB|      1.0|GRB|      1.0|
|GTF|      1.0|GTF|      1.0|
|IDA|      1.0|IDA|      1.0|
|GRR|      1.0|GRR|      1.0|
|LWB|      1.0|LWB|      1.0|
|JLN|      1.0|JLN|      1.0|
|PVU|      1.0|PVU|      1.0|
|EUG|      1.0|EUG|      1.0|
|PSG|      1.0|PSG|      1.0|
|ATY|      1.0|ATY|      1.0|
|GSO|      1.0|GSO|      1.0|
+---+---------+---+---------+
only showing top 20 rows



In [11]:
g.edges.groupBy('dst').count().orderBy('count', ascending=False).show()

+---+-----+
|dst|count|
+---+-----+
|ATL|32187|
|ORD|25687|
|DFW|24354|
|DEN|20405|
|CLT|20002|
|LAX|17794|
|PHX|15333|
|IAH|14793|
|LAS|14184|
|LGA|13836|
|SFO|13686|
|DTW|13190|
|MCO|12816|
|MSP|12428|
|BOS|12190|
|SEA|11585|
|DCA|11575|
|EWR|11420|
|JFK|10302|
|SLC|10213|
+---+-----+
only showing top 20 rows



In [12]:
g.edges.groupBy('src').count().orderBy('count', ascending=False).show()

+---+-----+
|src|count|
+---+-----+
|ATL|32190|
|ORD|25661|
|DFW|24339|
|DEN|20398|
|CLT|19995|
|LAX|17799|
|PHX|15325|
|IAH|14792|
|LAS|14186|
|LGA|13836|
|SFO|13684|
|DTW|13195|
|MCO|12820|
|MSP|12429|
|BOS|12182|
|SEA|11583|
|DCA|11575|
|EWR|11415|
|JFK|10312|
|SLC|10196|
+---+-----+
only showing top 20 rows



In [13]:
g.degrees.orderBy('degree', ascending=False).show(3)

[Stage 31:==============>                                           (2 + 6) / 8]

+---+------+
| id|degree|
+---+------+
|ATL| 64377|
|ORD| 51348|
|DFW| 48693|
+---+------+
only showing top 3 rows



In [15]:
distance_greater = g.edges.select('src', 'dst', 'DISTANCE').filter('DISTANCE > 500').distinct().orderBy('DISTANCE', ascending=False)
distance_greater.show()

[Stage 33:====================================>                     (5 + 3) / 8]

+---+---+--------+
|src|dst|DISTANCE|
+---+---+--------+
|BOS|HNL|  5095.0|
|HNL|BOS|  5095.0|
|JFK|HNL|  4983.0|
|HNL|JFK|  4983.0|
|EWR|HNL|  4962.0|
|HNL|EWR|  4962.0|
|IAD|HNL|  4817.0|
|HNL|IAD|  4817.0|
|ATL|HNL|  4502.0|
|HNL|ATL|  4502.0|
|DTW|HNL|  4475.0|
|HNL|DTW|  4475.0|
|HNL|ORD|  4243.0|
|ORD|HNL|  4243.0|
|ORD|OGG|  4184.0|
|OGG|ORD|  4184.0|
|MSP|HNL|  3972.0|
|HNL|MSP|  3972.0|
|HNL|IAH|  3904.0|
|IAH|HNL|  3904.0|
+---+---+--------+
only showing top 20 rows



In [16]:
from pyspark.sql.functions import desc
distance = g.edges.select('src', 'dst', 'DISTANCE').distinct().sort(desc('DISTANCE'))
distance.show()

[Stage 35:=======>                                                  (1 + 7) / 8]

+---+---+--------+
|src|dst|DISTANCE|
+---+---+--------+
|BOS|HNL|  5095.0|
|HNL|BOS|  5095.0|
|HNL|JFK|  4983.0|
|JFK|HNL|  4983.0|
|HNL|EWR|  4962.0|
|EWR|HNL|  4962.0|
|IAD|HNL|  4817.0|
|HNL|IAD|  4817.0|
|ATL|HNL|  4502.0|
|HNL|ATL|  4502.0|
|HNL|DTW|  4475.0|
|DTW|HNL|  4475.0|
|HNL|ORD|  4243.0|
|ORD|HNL|  4243.0|
|ORD|OGG|  4184.0|
|OGG|ORD|  4184.0|
|HNL|MSP|  3972.0|
|MSP|HNL|  3972.0|
|HNL|IAH|  3904.0|
|IAH|HNL|  3904.0|
+---+---+--------+
only showing top 20 rows



In [17]:
import pyspark.sql.functions as F
sub = GraphFrame(g.vertices, distance)
r = sub.find("(a)-[ab]->(b); (b)-[bc]->(c); !(a)-[]->(c)").filter('c.id != a.id')
r2 = r.withColumn("sum_distance", r.ab.DISTANCE + r.bc.DISTANCE).groupby('a.id','c.id').max('sum_distance').sort(desc('max(sum_distance)')).show()


[Stage 55:==========================================>           (158 + 8) / 200]

+---+---+-----------------+
| id| id|max(sum_distance)|
+---+---+-----------------+
|EWR|JFK|           9945.0|
|JFK|EWR|           9945.0|
|GUM|BOS|           8896.0|
|BOS|GUM|           8896.0|
|GUM|JFK|           8784.0|
|JFK|GUM|           8784.0|
|EWR|GUM|           8763.0|
|GUM|EWR|           8763.0|
|GUM|IAD|           8618.0|
|IAD|GUM|           8618.0|
|GUM|ATL|           8303.0|
|ATL|GUM|           8303.0|
|GUM|DTW|           8276.0|
|DTW|GUM|           8276.0|
|ORD|GUM|           8044.0|
|GUM|ORD|           8044.0|
|BOS|ANC|           7872.0|
|ANC|BOS|           7872.0|
|GUM|MSP|           7773.0|
|MSP|GUM|           7773.0|
+---+---+-----------------+
only showing top 20 rows



In [18]:
sub2 = GraphFrame(g.vertices, distance)
result = sub2.find("(a)-[ab]->(b); (b)-[bc]->(c); !(a)-[]->(c)").filter("a.id = 'JFK'").filter("c.id = 'EWR'")
result.show(100)

+------------+------------------+------------+------------------+------------+
|           a|                ab|           b|                bc|           c|
+------------+------------------+------------+------------------+------------+
|[JFK, 12478]| [JFK, BUF, 301.0]|[BUF, 10792]| [BUF, EWR, 282.0]|[EWR, 11618]|
|[JFK, 12478]|[JFK, SLC, 1990.0]|[SLC, 14869]|[SLC, EWR, 1969.0]|[EWR, 11618]|
|[JFK, 12478]| [JFK, PWM, 273.0]|[PWM, 14321]| [PWM, EWR, 284.0]|[EWR, 11618]|
|[JFK, 12478]|[JFK, STT, 1623.0]|[STT, 15024]|[STT, EWR, 1634.0]|[EWR, 11618]|
|[JFK, 12478]| [JFK, SYR, 209.0]|[SYR, 15096]| [SYR, EWR, 195.0]|[EWR, 11618]|
|[JFK, 12478]|[JFK, PBI, 1028.0]|[PBI, 14027]|[PBI, EWR, 1023.0]|[EWR, 11618]|
|[JFK, 12478]| [JFK, BNA, 765.0]|[BNA, 10693]| [BNA, EWR, 748.0]|[EWR, 11618]|
|[JFK, 12478]| [JFK, DCA, 213.0]|[DCA, 11278]| [DCA, EWR, 199.0]|[EWR, 11618]|
|[JFK, 12478]| [JFK, DTW, 509.0]|[DTW, 11433]| [DTW, EWR, 488.0]|[EWR, 11618]|
|[JFK, 12478]| [JFK, CLE, 425.0]|[CLE, 11042]| [CLE,